# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [21]:
DATA_FOLDER = 'ebola/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [22]:
# Write your answer here
import glob
import pandas as pd
import numpy as np
import os

def import_from_forder(folder_path):
    allFiles = glob.glob(folder_path + '/*.csv')
    frame = pd.DataFrame()
    list_ = []

    for file_ in allFiles:
        df = pd.read_csv(file_, header=0)
        list_.append(df)
    frame = pd.concat(list_)
    return frame

In [39]:
folders = [x[0] for x in os.walk(DATA_FOLDER)][1:]

#dataframes from 3 different folders
df1,df2,df3 = [import_from_forder(x) for x in folders]

# put date and description and total first (it was just for vision, it's useless)
df1 = df1[['Date', 'Description', 'Beyla', 'Boffa', 'Conakry', 'Coyah', 'Dabola', 
       'Dalaba', 'Dinguiraye', 'Dubreka', 'Forecariah', 'Gueckedou',
       'Kerouane', 'Kindia', 'Kissidougou', 'Kouroussa', 'Lola', 'Macenta',
       'Mzerekore', 'Nzerekore', 'Pita', 'Siguiri', 'Telimele', 
        'Yomou']]

df2 = df2[['Date', 'Variable', 'Bomi County', 'Bong County','Gbarpolu County', 'Grand Bassa',
       'Grand Cape Mount', 'Grand Gedeh', 'Grand Kru', 'Lofa County',
       'Margibi County', 'Maryland County', 'Montserrado County', 
       'Nimba County', 'River Gee County', 'RiverCess County', 'Sinoe County',
       'Unnamed: 18']]

df3 = df3[['date', 'variable', '34 Military Hospital', 'Bo', 'Bo EMC', 'Bombali', 'Bonthe',
       'Hastings-F/Town', 'Kailahun', 'Kambia', 'Kenema', 'Kenema (IFRC)',
       'Kenema (KGH)', 'Koinadugu', 'Kono', 'Moyamba',
       'Police training School', 'Police traning School', 'Port Loko',
       'Pujehun', 'Tonkolili', 'Unnamed: 18', 'Western area',
       'Western area combined', 'Western area rural', 'Western area urban']]


# get same notation for the 3 dataframes
df2 = df2.rename(columns = {'Variable':'Description'})
df3 = df3.rename(columns = {'date':'Date', 'variable':'Description'})

In [38]:
# concatenate the three dataframes and reorder columns
df = pd.concat([df1,df2,df3])
df = df[['Date', 'Description', 'Totals', 'Totals2', 'Totals3', '34 Military Hospital', 'Beyla', 'Bo', 'Bo EMC', 'Boffa', 'Bombali',
       'Bomi County', 'Bong County', 'Bonthe', 'Conakry', 'Coyah', 'Dabola',
       'Dalaba', 'Dinguiraye', 'Dubreka', 'Forecariah',
       'Gbarpolu County', 'Grand Bassa', 'Grand Cape Mount', 'Grand Gedeh',
       'Grand Kru', 'Gueckedou', 'Hastings-F/Town', 'Kailahun', 'Kambia',
       'Kenema', 'Kenema (IFRC)', 'Kenema (KGH)', 'Kerouane', 'Kindia',
       'Kissidougou', 'Koinadugu', 'Kono', 'Kouroussa', 'Lofa County', 'Lola',
       'Macenta', 'Margibi County', 'Maryland County', 'Montserrado County',
       'Moyamba', 'Mzerekore', 'Nimba County', 'Nzerekore', 'Pita',
       'Police training School', 'Police traning School', 'Port Loko',
       'Pujehun', 'River Gee County', 'RiverCess County', 'Siguiri',
       'Sinoe County', 'Telimele', 'Tonkolili', 'Unnamed: 18',
       'Western area', 'Western area combined', 'Western area rural',
       'Western area urban', 'Yomou']]


Totals = df.loc[:, 'Date':'Totals3']
Totals

,Date,Description,Totals,Totals2,Totals3
0,2014-08-04,New cases of suspects,5,NaN,NaN
1,2014-08-04,New cases of probables,0,NaN,NaN
2,2014-08-04,New cases of confirmed,4,NaN,NaN
3,2014-08-04,Total new cases registered so far,9,NaN,NaN
4,2014-08-04,Total cases of suspects,11,NaN,NaN
5,2014-08-04,Total cases of probables,133,NaN,NaN
6,2014-08-04,Total cases of confirmed,351,NaN,NaN
7,2014-08-04,Cumulative (confirmed + probable + suspects),495,NaN,NaN
8,2014-08-04,New deaths registered today,2,NaN,NaN
9,2014-08-04,New deaths registered today (confirmed),2,NaN,NaN


In [33]:
df_new_cases = df.loc[df.Description.str.contains('new') & df.Description.str.contains('case')]
df_death = df.loc[df.Description.str.contains('new') & df.Description.str.contains('death')]
df_new_cases.Description.value_counts()

new_noncase                          103
Total new cases registered so far     22
Name: Description, dtype: int64

In [17]:
test = df[(df['Date'] > '2014-08-01') & (df.Date < '2014-09-01')]
test = test.loc[test.Description.str.contains('new') & test.Description.str.contains('death')]
pd.to_datetime(test['Date'])
test.groupby(test.Date).mean()

,34 Military Hospital,Bo EMC,Bomi County,Bong County,Gbarpolu County,Grand Bassa,Grand Cape Mount,Grand Gedeh,Grand Kru,Hastings-F/Town,...,Montserrado County,Nimba County,Police training School,Police traning School,River Gee County,RiverCess County,Sinoe County,Unnamed: 18,Western area,Western area combined
Date,,,,,,,,,,,,,,,,,,,,,
2014-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# nothing that works...
dataframe = frame
dataframe["test"] = dataframe["Date"].fillna(dataframe["date"])
frame.loc[frame.Totals.isnull()]
description = pd.DataFrame({'Description': pd.concat([frame.Variable, frame.variable, frame.Description])}).sort_index()
date = pd.DataFrame({'Date' : pd.concat([frame.Date, frame.date])}).sort_index()

NameError: name 'frame' is not defined

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here